# Using Basic Phases

All simulations in OpenPNM eventually require knowing the physical and thermodynamic properties of the fluids and phases.  OpenPNM provides a framework for computing these values in the ``phase`` submodule, along with a *basic* set of functions for predicting the properties pure phases and mixtures. Since we can't possibly support an exhaustive physical proprety library, our policy is to provide a set of reasonable default functions for use as first approximations.  There are a number of other packages that can be used when more exacting values are required, such as ``chemicals`` and ``cantera``, which can be used within the framework of OpenPNM.

In the first part of this notebook, we will be cover: 
* The use of the basic ``Phase`` class
* Assigning constant values
* Generating values using built-in pore-scale models
* Creating custom pore-scale models
* Specific classes for common fluids like ``Air`` and ``Water``

In the second part we will cover topics relating to Mixtures, including:
* Creating individual ``Species`` objects from the CAS number
* Combining components into a ``Mixture``
* Utilizing the built-in classes for first-approximations of pure gas and liquid properties, and for mixture properties
* Leveraging external packages for making better estimates of pure component and mixture properties

In [2]:
import openpnm as op
import numpy as np

All simulations start by defining/creating a network.  The ``Demo`` class creates a very simple cubic network with an assortment of useful properties included:

In [3]:
pn = op.network.Demo()

## Part 1: The ``Phase`` class

If your simulation is simple, then a simple ``Phase`` object may be sufficient.  It has no predefined models from computing anything, so you have to either assign known values directly (e.g ``water['pore.viscosity'] = 0.001``) or define models that will compute the values you need.  The models can be taken from the ``openpnm.models.phase`` library, or you can write your own.

In [4]:
phase1 = op.phase.Phase(network=pn)
print(phase1)


══════════════════════════════════════════════════════════════════════════════
phase_01 : <openpnm.phase.Phase at 0x2320e778220>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.pressure                                                       9 / 9
  2  pore.temperature                                                    9 / 9
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Labels                                                 Assigned Locations
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.all                                                                9
  2  throat.all                                                             12
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――


Note that all ``phase`` objects need to be associated with a network, which is how ``phase1`` knows how many pore (and throat) values to compute...in this case 9 (and 12). 

### Direct assignment of a constant value

The basic ``Phase`` class creates a more or less empty object with only standard temperature and pressure assigned to each pore. In order to use this object for simulations it needs some additional information. For instance, to compute the permeability of ``pn`` we will need the viscosity.  So, let's assign a known value directly for liquid water:

In [4]:
phase1['pore.viscosity'] = 0.001  # Pa.s
phase1['pore.temperature'] = 273.0

*Pro Tip*: When assigning a scalar value to a dictionary key it gets assigned to every pore (or throat).  The result of the above assignment can be seen below:

In [5]:
phase1['pore.viscosity']

array([0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001])

### Using a built-in model

Perhaps you would like to run simulation in the presence of a temperature gradient, and viscosity is a strong function of temperature.  Instead of assigning a constant viscosity, in this case it is better to assign a pore-scale model that OpenPNM will call to compute the viscosity in each pore.  

The ``models`` library in OpenPNM contains some general models which can be used, such as polynomials or linear lines. A 4th order polynomial can be fit to experimental data yielding the following coefficients:

In [6]:
a4 = 5.8543E-11
a3 = -7.6756E-08
a2 = 3.7831E-05
a1 = -8.3156E-03
a0 = 6.8898E-01

These can be used in the the ``op.models.misc.polynomial`` model as follows:

In [15]:
print(phase1['pore.viscosity'])
f = op.models.misc.polynomial
phase1.add_model(propname='pore.viscosity', 
                 model=f,
                 a = (a0, a1, a2, a3, a4),
                 prop='pore.temperature')
print(phase1['pore.viscosity'])
phase1['pore.temperature'] = 300.0 + np.random.rand(pn.Np)*50
phase1.regenerate_models()
print(phase1['pore.viscosity'])

[0.00179952 0.00179952 0.00179952 0.00179952 0.00179952 0.00179952
 0.00179952 0.00179952 0.00179952]
[0.00078261 0.00047368 0.0006127  0.00042288 0.00066796 0.00083404
 0.00047012 0.00059282 0.00056671]
[0.00067764 0.00042716 0.00087339 0.0004289  0.00086183 0.00044696
 0.00065252 0.0004201  0.00069108]


In the above block we can see that the values of ``0.001`` are present from the previous assignment, then after we add the model the values are recomputed at the temperature indicated in ``phase1['pore.temperature']``.  To illustrate the point of using a temperature dependent model, we then set the temperature to a random number between 300 and 350 K, the rerun the models at the new temperatures.

### Using a water-specific model

Because water is so common, OpenPNM has some available functions for its properties:

In [16]:
f = op.models.phase.viscosity.water_correlation
phase1.add_model(propname='pore.viscosity',
                 model=f)
print(phase1['pore.viscosity'])

[0.000647   0.00039139 0.00085086 0.0003938  0.00083897 0.00041547
 0.00062088 0.00038031 0.00066103]


### Writing your own custom model

This subject is explained in detail in another tutorial, but the basic outline is as follows. 

In [17]:
def custom_mu(target, temperature='pore.temperature'):
    T = target[temperature]
    a4 = 5.854E-11
    a3 = -7.676E-08
    a2 = 3.783E-05
    a1 = -8.316E-03
    a0 = 6.890E-01
    mu = a0 + a1*T + a2*T**2 + a3*T**3 + a4*T**4
    return mu

phase1.add_model(propname='pore.viscosity',
                 model=custom_mu)
print(phase1['pore.viscosity'])


[ 3.21402876e-04 -1.80439511e-05  5.50708547e-04 -1.49215969e-05
  5.37611746e-04  1.48067041e-05  2.90493860e-04 -3.16812236e-05
  3.37779603e-04]


## Specific Classes for Common Fluids

Air, water, and mercury are used commonly enough that OpenPNM not only has pore-scale models for their propertie (i.e. ``op.models.viscosity.water_correlation``, but we have also created pre-defined classes with all the appropriate models already attached:

In [18]:
water = op.phase.Water(network=pn)

In [19]:
print(water)


══════════════════════════════════════════════════════════════════════════════
phase_03 : <openpnm.phase.Water at 0x29032055e50>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.contact_angle                                                  9 / 9
  2  pore.density                                                        9 / 9
  3  pore.molar_density                                                  9 / 9
  4  pore.pressure                                                       9 / 9
  5  pore.surface_tension                                                9 / 9
  6  pore.temperature                                                    9 / 9
  7  pore.thermal_conductivity                                           9 / 9
  8  pore.vapor_pressure                                                 9 / 9
 

As can be seen in the above print-out, a variety of things have been computed, most of which is coming from a specific pore-scale model.  These can be viewed with:

In [20]:
print(water.models)

―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
#   Property Name                       Parameter                 Value
―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
1   pore.contact_angle@all              model:                    constant
                                        value:                    110.0
                                        regeneration mode:        normal
―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
2   pore.density@all                    model:                    water_correlation
                                        T:                        pore.temperature
                                        salinity:                 pore.salinity
                                        regeneration mode:        normal
―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
3   pore.molar_density@all              mode

All of these models are functions of all relevant properties, so we can change the temperature and see the new viscosity:

In [22]:
print(water['pore.viscosity'])
water['pore.temperature'] = 300.0 + np.random.rand(pn.Np)*50
water.regenerate_models()
print(water['pore.viscosity'])

[0.0005704  0.00062362 0.00041976 0.00053435 0.00062575 0.00080914
 0.0004782  0.000501   0.00038744]
[0.00042246 0.00037174 0.00071614 0.0003947  0.00047906 0.0005611
 0.00037583 0.00041211 0.00075005]


## Part 2: Using Species and Mixtures

OpenPNM provides a small set of default function for computing the physical properties of pure fluids as well as mixtures.  This notebook will cover their use

### Pure Species

In [5]:
A = op.phase.Species(network=pn, species='ethanol')
print(A)


══════════════════════════════════════════════════════════════════════════════
phase_02 : <openpnm.phase.Species at 0x23207ff65e0>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.pressure                                                       9 / 9
  2  pore.temperature                                                    9 / 9
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Labels                                                 Assigned Locations
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.all                                                                9
  2  throat.all                                                             12
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

As can be seen above the ``Species`` class does not compute any properties of the given species, *BUT* it does contain a host of thermodynamic properties in the ``params`` attribute:

In [6]:
print(A.params)

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Parameters                          Value
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
CAS                                 64-17-5
common_name                         ethanol
charge                              0
formula                             C2H6O
boiling_temperature                 351.39
melting_temperature                 159.05
triple_point_temperature            150.0
triple_point_pressure               0.0003029719071725671
dipole_moment                       1.44
LJ_diameter                         4.23738
LJ_energy                           1.7829839250900002e-20
surface_tension_Tb                  0.016709805941752066
molar_volume_Tb                     6.269082962629869e-05
molecular_weight                    46.06844
critical_temperature                514.0
critical_pressure                   6137000.0
critical_volume                     0.000168
critical_

----
💡 **Pro Tip**: You can access the values of these parameters either from the ``params`` attribute or using the dictionary lookup of the main object which will dispatch the query to the ``params`` attribute. 

In [43]:
water.params['CAS']

'7732-18-5'

In [45]:
water['param.CAS']

'7732-18-5'

Writing also works.

In [46]:
water['param.foo'] = 'bar'
water.params['foo']

'bar'

----

These parameters are all used in the various property estimation methods.  For instance, to compute the viscosity of ethanol, OpenPNM provides a function that implements the model of Steil and Thodos (``openpnm.models.phase.viscosity.liquid_pure_ls``):

In [7]:
f = op.models.phase.viscosity.liquid_pure_ls
A.add_model(propname='pore.viscosity',
            model=f)
print(A['pore.viscosity'])

[0.00041951 0.00041951 0.00041951 0.00041951 0.00041951 0.00041951
 0.00041951 0.00041951 0.00041951]


This function requires several pieces of thermodynamics information, such as the critical temperature and pressure. You can see all the arguments below:

In [8]:
print(A.models)

―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
#   Property Name                       Parameter                 Value
―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
1   pore.viscosity@all                  model:                    liquid_pure_ls
                                        T:                        pore.temperature
                                        MW:                       param.molecular_weight
                                        Tc:                       param.critical_temperature
                                        Pc:                       param.critical_pressure
                                        omega:                    param.acentric_factor
                                        regeneration mode:        normal
―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――


The above shows that the temperature of the phase is fetched as ``'pore.temperature'``, while all the rest are retrieved from the ``params`` attribute.  To further illustrate this behavior, we could write the critical temperature ``A['pore.critical_temperature']`` and also overwrite the default argument:

In [9]:
A.models['pore.viscosity@all']['Tc'] = 'pore.critical_temperature'
A['pore.critical_temperature'] = A['param.critical_temperature']

In [10]:
print(A.models)

―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
#   Property Name                       Parameter                 Value
―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
1   pore.viscosity@all                  model:                    liquid_pure_ls
                                        T:                        pore.temperature
                                        MW:                       param.molecular_weight
                                        Tc:                       pore.critical_temperature
                                        Pc:                       param.critical_pressure
                                        omega:                    param.acentric_factor
                                        regeneration mode:        normal
―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――


Now when we regenerate the model it will fetch the critical temperature values for each pore but will work as expected:

In [11]:
A.regenerate_models()
print(A['pore.viscosity'])

[0.00041951 0.00041951 0.00041951 0.00041951 0.00041951 0.00041951
 0.00041951 0.00041951 0.00041951]


### Gas and Liquid Species

OpenPNM has a suite of functions for computing the properties of pure phases, but these functions differ for gases and liquids.  For this reason we offer two classes for gas and liquid with the appropriate models already defined.  These are referred to as ``StandardLiquid`` and ``StandardGas`` to indicate that the models being used are the standard selection which provide a first-approximatation:

In [12]:
A = op.phase.StandardGas(network=pn, species='o2')
B = op.phase.StandardLiquid(network=pn, species='h2o')

These objects are populated with their respective thermodynamic properties:

In [13]:
print(A.params)

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Parameters                          Value
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
CAS                                 7782-44-7
common_name                         oxygen
charge                              0
formula                             O2
boiling_temperature                 90.188
melting_temperature                 54.36
triple_point_temperature            54.33
triple_point_pressure               148.9796864589355
dipole_moment                       0.0
LJ_diameter                         3.29728
LJ_energy                           1.6520845934e-21
surface_tension_Tb                  0.013145633010272155
molar_volume_Tb                     2.802254619756072e-05
molecular_weight                    31.9988
critical_temperature                154.58
critical_pressure                   5042945.25
critical_volume                     7.34e-05
critical_compressibilt

The models used on each can be seen by printing the models attribute:

In [14]:
print(A)


══════════════════════════════════════════════════════════════════════════════
phase_03 : <openpnm.phase.StandardGas at 0x2320e778180>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.density                                                        9 / 9
  2  pore.heat_capacity                                                  9 / 9
  3  pore.heat_capacity_gas                                              9 / 9
  4  pore.pressure                                                       9 / 9
  5  pore.temperature                                                    9 / 9
  6  pore.thermal_conductivity                                           9 / 9
  7  pore.viscosity                                                      9 / 9
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

In [15]:
print(B)


══════════════════════════════════════════════════════════════════════════════
phase_04 : <openpnm.phase.StandardLiquid at 0x23213344ef0>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.density                                                        9 / 9
  2  pore.heat_capacity                                                  9 / 9
  3  pore.heat_capacity_gas                                              9 / 9
  4  pore.pressure                                                       9 / 9
  5  pore.temperature                                                    9 / 9
  6  pore.thermal_conductivity                                           9 / 9
  7  pore.vapor_pressure                                                 9 / 9
  8  pore.viscosity                                                    

### Gas and Liquid Mixtures

The framework for defining mixtures was officially added in V3, and it allows for the calculation of mixture properties based on the properties of the individual phases and their relative concentrations. There are a number of pore-scale models added to the ``openpnm.models.phase`` library for making these estimations, either using mixing rules or other correlations.

Creating a mixture first requires defining each of the pure components.  In this case, let's create vodka:

In [17]:
water = op.phase.StandardLiquid(network=pn, species='h2o')
etoh = op.phase.StandardLiquid(network=pn, species='ethanol')

Next let's create the mixture:

In [18]:
vodka = op.phase.StandardLiquidMixture(network=pn, components=[water, etoh])

Before using this we must first specify the compositions:

In [19]:
vodka.x(water, 0.6)
vodka.x(etoh, 0.4)

Now we can run the models, which all require knowing the composition:

In [20]:
vodka.regenerate_models()

The main class is the ``Mixture`` class, and there are several subclasses which each add some specific functionality

The printout of a mixture includes not only the mixture properties, but also shows the individual components as well:

In [22]:
f = vodka.models['pore.heat_capacity@all']['model']

We can see that several properties for the mixture have been computed:

In [23]:
print(vodka)


══════════════════════════════════════════════════════════════════════════════
mixture_01 : <openpnm.phase.StandardLiquidMixture at 0x2321336c720>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.density                                                        9 / 9
  2  pore.heat_capacity                                                  9 / 9
  3  pore.mole_fraction.phase_07                                         9 / 9
  4  pore.mole_fraction.phase_08                                         9 / 9
  5  pore.pressure                                                       9 / 9
  6  pore.temperature                                                    9 / 9
  7  pore.thermal_conductivity                                           9 / 9
  8  pore.viscosity                                           

If we change the composition of the components, and rerun the models, the mixture properties will change:

In [24]:
print(vodka['pore.viscosity'])
vodka.x(water, 0.8)
vodka.x(etoh, 0.2)
vodka.regenerate_models()
print(vodka['pore.viscosity'])

[0.00053084 0.00053084 0.00053084 0.00053084 0.00053084 0.00053084
 0.00053084 0.00053084 0.00053084]
[0.00057417 0.00057417 0.00057417 0.00057417 0.00057417 0.00057417
 0.00057417 0.00057417 0.00057417]


----
💡 **ProTip!** The mole fractions of *both* species can be retrieved using the `dict` lookup without specifying which component.  OpenPNM will return a dictionary of both components with their names as they keys:

In [47]:
print(vodka['pore.mole_fraction'])

{'phase_07': array([0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6]), 'phase_08': array([0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4])}


💡 This also means you can index into the returned dictionary using the names:

In [48]:
print(vodka['pore.mole_fraction'][water.name])

[0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6]


💡 Alternatively the returned dictionary can be used to get a list of components that are currently part of the mixture:

In [49]:
vodka['pore.mole_fraction'].keys()

dict_keys(['phase_07', 'phase_08'])

💡 The individual species can be retrieved using the ``components`` attribute which returns a dictionary with component names as the keys and handles to the actual objects as values:

In [50]:
vodka.components

{'phase_07': phase_07 : <openpnm.phase.StandardLiquid at 0x232133376d0>,
 'phase_08': phase_08 : <openpnm.phase.StandardLiquid at 0x2321336cf40>}

---

### Getting and Setting Compositions

The ``x`` method (and ``y`` on ``GasMixture``) can also be used to retrieve the mole fraction of one or both components:

In [51]:
vodka.x()

{'phase_07': array([0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6]),
 'phase_08': array([0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4])}

In [54]:
vodka.x(water.name)

In addition to keeping track of the species and their compositions, the mixture object also computes mixture properties. These can be seen by printing ``air``.  Note that we must first called ``regenerate_models`` since the models were not run on instantiation since the mole fraction was not set:

In [55]:
vodka.regenerate_models()
print(vodka)


══════════════════════════════════════════════════════════════════════════════
mixture_01 : <openpnm.phase.StandardLiquidMixture at 0x2321336c720>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.density                                                        9 / 9
  2  pore.heat_capacity                                                  9 / 9
  3  pore.mole_fraction.phase_07                                         9 / 9
  4  pore.mole_fraction.phase_08                                         9 / 9
  5  pore.pressure                                                       9 / 9
  6  pore.temperature                                                    9 / 9
  7  pore.thermal_conductivity                                           9 / 9
  8  pore.viscosity                                           

Several of the properties listed above were on the species objects, such as viscosity.  These viscosity of the mixture is computed using a mole-weighted approach. The molecular weight is also the mole-weighted average of the two species.  

In [58]:
vodka['pore.viscosity']

array([0.00053084, 0.00053084, 0.00053084, 0.00053084, 0.00053084,
       0.00053084, 0.00053084, 0.00053084, 0.00053084])

### Exploring the Features of the Mixture Objects

#### Adding and Removing Species

You can remove a species:

In [38]:
vodka.remove_comp(water.name)
vodka.components

{'phase_08': phase_08 : <openpnm.phase.StandardLiquid at 0x2321336cf40>}

----
💡 **ProTip!** A species is considered a component of a mixture *if and only if* ``'pore.mole_fraction.<species.name>'`` appears in the mixture dictionary. Adding and removing the corresponding array from the dictionary is literally how the components are defined. For instance:

In [39]:
del vodka['pore.mole_fraction.' + etoh.name]
vodka.components

{}

They can be readded in the same way:

In [40]:
vodka['pore.mole_fraction.' + water.name] = 0.60
vodka.components

{'phase_07': phase_07 : <openpnm.phase.StandardLiquid at 0x232133376d0>}

But there is a specific method for this:

In [41]:
vodka.add_comp(etoh, mole_fraction=0.40)
vodka.components

{'phase_07': phase_07 : <openpnm.phase.StandardLiquid at 0x232133376d0>,
 'phase_08': phase_08 : <openpnm.phase.StandardLiquid at 0x2321336cf40>}

#### ``info``
The info attribute reports all the existing properties on the mixture (similar to ``print``) but also of each of the components:

In [25]:
vodka.info


══════════════════════════════════════════════════════════════════════════════
mixture_01 : <openpnm.phase.StandardLiquidMixture at 0x2321336c720>
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  #  Properties                                                   Valid Values
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
  1  pore.density                                                        9 / 9
  2  pore.heat_capacity                                                  9 / 9
  3  pore.mole_fraction.phase_07                                         9 / 9
  4  pore.mole_fraction.phase_08                                         9 / 9
  5  pore.pressure                                                       9 / 9
  6  pore.temperature                                                    9 / 9
  7  pore.thermal_conductivity                                           9 / 9
  8  pore.viscosity                                           

#### Get component mole fractions

You'll notice that the mixture object has arrays called ``'pore.mole_fraction.<compname>'`` for each component. The dictionary look-up in OpenPNM will return a subdictionary if the given key is just ``'pore.mole_fraction'``.

In [26]:
vodka['pore.mole_fraction']

{'phase_07': array([0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8, 0.8]),
 'phase_08': array([0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2])}

#### ``components``
This attribute returns a dictionary with each of the components accessible by their name:

In [27]:
d = vodka.components
print(d.keys())

dict_keys(['phase_07', 'phase_08'])


#### ``get_comp_vals``
Since the mixture is made from several components, it is often desired get the values of a specific property from each component.  This method provides a convenient way to do this:

In [28]:
mus = vodka.get_comp_vals('pore.viscosity')
print(mus)

{'phase_07': array([0.00062103, 0.00062103, 0.00062103, 0.00062103, 0.00062103,
       0.00062103, 0.00062103, 0.00062103, 0.00062103]), 'phase_08': array([0.00041951, 0.00041951, 0.00041951, 0.00041951, 0.00041951,
       0.00041951, 0.00041951, 0.00041951, 0.00041951])}


It is also possible to retrieve the properteis of a component by asking the mixture and appending the component name, as follows:

In [30]:
vodka['pore.viscosity.' + water.name]

array([0.00062103, 0.00062103, 0.00062103, 0.00062103, 0.00062103,
       0.00062103, 0.00062103, 0.00062103, 0.00062103])

In reality there is no array on ``vodka`` with the name ``'pore.viscosity.phase_01'``, but failure to find this array is what actually triggers the look-up of the array from ``water``. This is a convenient feature that is added using some 'syntactic sugar' behind the scenes in Python.  

#### ``check_mixture_health``

You can also check the health of the mixture, such as whether all the mole fractions add to 1.0 each each pore:

In [35]:
print(vodka.check_mixture_health())

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Key                                 Value
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
mole_fraction_too_low               []
mole_fraction_too_high              []
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――


In [37]:
vodka.x(water.name, 0.1)
print(vodka.check_mixture_health())

――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
Key                                 Value
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――
mole_fraction_too_low               (9,)
mole_fraction_too_high              []
――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――


#### Retrieving Species Properties

Lastly, the properties of the individual species can be retrieved from the mixture as follows:

In [42]:
vodka.get_comp_vals('pore.viscosity')

{'phase_07': array([0.00062103, 0.00062103, 0.00062103, 0.00062103, 0.00062103,
        0.00062103, 0.00062103, 0.00062103, 0.00062103]),
 'phase_08': array([0.00041951, 0.00041951, 0.00041951, 0.00041951, 0.00041951,
        0.00041951, 0.00041951, 0.00041951, 0.00041951])}

#### Using the wildcard (``*``) syntax

One more feature that has been added is the ability to fetch requested property arrays from *all* the components by replacing the component name with the universal *wildcard* symbmol: ``*`` as follows:

In [31]:
vodka['pore.viscosity.*']

{'phase_07': array([0.00062103, 0.00062103, 0.00062103, 0.00062103, 0.00062103,
        0.00062103, 0.00062103, 0.00062103, 0.00062103]),
 'phase_08': array([0.00041951, 0.00041951, 0.00041951, 0.00041951, 0.00041951,
        0.00041951, 0.00041951, 0.00041951, 0.00041951])}

Note that this returns exactly the same dictionary as the ``get_comp_vals`` method (in fact this function gets called behind the scenes), but this feature is offered for more than just convenience.  The main reason for supporting this feature is so that pore-scale models can be instructed to fetch the needed arrays for computing the mixture properties.  This is demonstrated in the following simple example of a custom mixture model:

In [32]:
def mole_fraction_weighting(phase, propname):
    xs = phase['pore.mole_fraction']
    ys = phase[propname]  # This is the key step
    z = 0.0
    for i in xs.keys():
        z += xs[i]*ys[i]
    return z

In [33]:
vals = mole_fraction_weighting(phase=vodka, propname='pore.viscosity.*')
print(vals)

[0.00058073 0.00058073 0.00058073 0.00058073 0.00058073 0.00058073
 0.00058073 0.00058073 0.00058073]


The use of the ``'.*'`` as the suffix of the ``propname`` argument is crucial here.  As can be seen in the definition of ``mole_fraction_weighting``, the call to ``phase[propname]`` passes ``'pore.viscsoity.*'`` directly to the dictionary lookup of values from ``phase``, and this in turn triggers the retrieval of the ``'pore.viscosity'`` values from each component. 

If we were to pass ``'pore.viscosity'`` then the function would throw an error since the call to ``phase['pore.viscosity']`` would return a single numpy array of viscosity values of the mixture (or not find any values at all).